In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHomilvTgLFwZ8ByuU0LJ2y8UIzeOYMA0iwv970Kc2BphoQyiw2NSU
Mounted at /content/drive


In [3]:
label_dic = {}
train_texts, train_labels = [], []
test_texts, test_labels = [], []

with open('/content/drive/My Drive/Colab Notebooks/data/news_sample/train.tsv', 'r', encoding='utf-8') as fr:
  lines = fr.readlines()
  for line in lines:
    line = line.strip().split('\t')
    label = line[1]
    title = line[3]
    content = line[4]
    
    if label not in label_dic.keys():
      label_dic[label] = len(label_dic)

    train_texts.append(title + '\n' + content)
    train_labels.append(label_dic[label])

with open('/content/drive/My Drive/Colab Notebooks/data/news_sample/test.tsv', 'r', encoding='utf-8') as fr:
  lines = fr.readlines()
  for line in lines:
    line = line.strip().split('\t')
    label = line[1]
    title = line[3]
    content = line[4]
  
    test_texts.append(title + '\n' + content)
    test_labels.append(label_dic[label])

In [4]:
print(label_dic)

{'IT과학': 0, '정치': 1, 'economy': 2}


In [10]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import numpy as np
import tensorflow as tf

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
class Tokenizer:

  def make_vocab(self, documents):
    for document in documents:
      tokens = self.tokenize(document)
      word2index = {'<unk>':0}
      for voca in tokens:
        if voca not in word2index.keys():
          word2index[voca] = len(word2index)
    self.vocab = word2index

  def tokenize(self, document):
    words = []
    sentences = sent_tokenize(document)

    for sentence in sentences:
      words.extend(WordPunctTokenizer().tokenize(sentence))

    return words

  def one_hot_encoding(self, word):
    one_hot_vector = [0] * len(self.vocab)
    if word not in self.vocab:
      word = '<unk>'
    index = self.vocab[word]
    one_hot_vector[index] = 1
    return one_hot_vector
  
  def get_vector(self, sentence):
    tokens = self.tokenize(sentence)
    vector = [self.one_hot_encoding(token) for token in tokens]

    while len(vector) < 256:
      vector.append([0] * len(self.vocab))
    
    return vector[:256]

In [12]:
tokenizer = Tokenizer()
tokenizer.make_vocab(train_texts)

In [16]:
x_train = np.array([tokenizer.get_vector(text) for text in train_texts])
x_test = np.array([tokenizer.get_vector(text) for text in test_texts])

y_train = np.array(train_labels)
y_test = np.array(test_labels)

print(x_train.shape, x_test.shape)

(450, 256, 340) (50, 256, 340)


In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 256)               480256    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 496,899
Trainable params: 496,899
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
15/15 [==============================] - 11s 725ms/step - loss: 1.0685 - accuracy: 0.4267
Epoch 2/20
15/15 [==============================] - 11s 715ms/step - loss: 1.0414 - accuracy: 0.4511
Epoch 3/20
15/15 [==============================] - 11s 713ms/step - loss: 1.0290 - accuracy: 0.4556
Epoch 4/20
15/15 [==============================] - 11s 719ms/step - loss: 1.0242 - accuracy: 0.4911
Epoch 5/20
15/15 [==============================] - 11s 712ms/step - loss: 0.9949 - accuracy: 0.4911
Epoch 6/20
14/15 [===========================>..] - ETA: 0s - loss: 0.9684 - accuracy: 0.5357

In [ ]:
model.evaluate(x_test, y_test, verbose=2)